## 🏢🧩 Step 1: Exhibition Center Data Modeling

### Import Libraries

In [1]:
import osmnx as ox # to fetch data from OpenStreetMap
import geopandas as gpd # to work with geospatial data
import pandas as pd
import numpy as np
import re
from geopy.geocoders import Nominatim
from tqdm import tqdm

### Create the tag to pull galleries from tourism section

In [2]:
tags = {
        "amenity": "exhibition_centre"
       }

### Fetch Berlin Geometries

In [3]:
exhibition_centers_raw = ox.features_from_place("Berlin, Germany", tags)

### Display basic info

In [4]:
print(f"Number of exhibition center entries fetched: {len(exhibition_centers_raw)}")
exhibition_centers_raw.head()

Number of exhibition center entries fetched: 4


geometry  \
element id                                                             
way     136079469  POLYGON ((13.26652 52.5022, 13.26672 52.50183,...   
        196694606  POLYGON ((13.2704 52.50082, 13.27008 52.50056,...   
        680746621  POLYGON ((13.26754 52.50363, 13.26646 52.50277...   
        719414532  POLYGON ((13.57087 52.53925, 13.57083 52.53926...   

                             amenity     landuse                  name  \
element id                                                               
way     136079469  exhibition_centre  commercial          Messe Berlin   
        196694606  exhibition_centre         NaN       CityCube Berlin   
        680746621  exhibition_centre         NaN                 hub27   
        719414532  exhibition_centre         NaN  Ausstellungspavillon   

                                                             website  \
element id                                                             
way     136079469                       https://www.messe-berlin.de/   
        196694606                                                NaN   
        680746621  https://www.messe-berlin.de/de/veranstalter/un...   
        719414532                                                NaN   

                     wikidata      wikimedia_commons  \
element id                                             
way     136079469     Q708065  Category:Messe Berlin   
        196694606   Q15108815                    NaN   
        680746621  Q116290308                    NaN   
        719414532         NaN                    NaN   

                                  wikipedia addr:city addr:country  ...  \
element id                                                          ...   
way     136079469  de:Messegelände (Berlin)       NaN          NaN  ...   
        196694606        de:CityCube Berlin    Berlin           DE  ...   
        680746621                       NaN    Berlin          NaN  ...   
        719414532                       NaN       NaN          NaN  ...   

                  addr:postcode  addr:street addr:suburb building  check_date  \
element id                                                                      
way     136079469           NaN          NaN         NaN      NaN         NaN   
        196694606         14055    Messedamm     Westend      yes  2024-07-04   
        680746621         14055  Jafféstraße     Westend      yes         NaN   
        719414532           NaN          NaN         NaN      yes  2025-03-31   

                                  contact:website layer          short_name  \
element id                                                                    
way     136079469                             NaN   NaN                 NaN   
        196694606  http://www.citycube-berlin.de/     2  CityCube;City Cube   
        680746621                             NaN   NaN                 NaN   
        719414532                             NaN   NaN                 NaN   

                     addr:place         alt_name  
element id                                        
way     136079469           NaN              NaN  
        196694606           NaN              NaN  
        680746621  Messe Berlin  hub 27;Halle 27  
        719414532           NaN              NaN  

[4 rows x 21 columns]

### Save the raw data to a csv file

- Define file paths

In [5]:
raw_csv_path = "../sources/csv_files/exhibition_center_raw.csv"
raw_geojson_path = "../sources/geojson_files/exhibition_center_raw.geojson"

- Save csv & geojson to correct folders in sources

In [6]:
exhibition_centers_raw.to_csv(raw_csv_path, index=False )

In [7]:
exhibition_centers_raw.to_file(raw_geojson_path, driver="GeoJSON")

### Remove the columns with 85% or more missing data

In [8]:
# Step 1: Calculate percentage of missing values per column
missing_percent = exhibition_centers_raw.isnull().mean() * 100

# Step 2: Identify columns with less than 85% missing
columns_to_keep = missing_percent[missing_percent < 85].index

# Step 3: Create a new filtered DataFrame
exhibition_centers_filtered = exhibition_centers_raw[columns_to_keep]

# Check shape and preview
print(exhibition_centers_filtered.shape)
exhibition_centers_filtered.head()

(4, 21)


geometry  \
element id                                                             
way     136079469  POLYGON ((13.26652 52.5022, 13.26672 52.50183,...   
        196694606  POLYGON ((13.2704 52.50082, 13.27008 52.50056,...   
        680746621  POLYGON ((13.26754 52.50363, 13.26646 52.50277...   
        719414532  POLYGON ((13.57087 52.53925, 13.57083 52.53926...   

                             amenity     landuse                  name  \
element id                                                               
way     136079469  exhibition_centre  commercial          Messe Berlin   
        196694606  exhibition_centre         NaN       CityCube Berlin   
        680746621  exhibition_centre         NaN                 hub27   
        719414532  exhibition_centre         NaN  Ausstellungspavillon   

                                                             website  \
element id                                                             
way     136079469                       https://www.messe-berlin.de/   
        196694606                                                NaN   
        680746621  https://www.messe-berlin.de/de/veranstalter/un...   
        719414532                                                NaN   

                     wikidata      wikimedia_commons  \
element id                                             
way     136079469     Q708065  Category:Messe Berlin   
        196694606   Q15108815                    NaN   
        680746621  Q116290308                    NaN   
        719414532         NaN                    NaN   

                                  wikipedia addr:city addr:country  ...  \
element id                                                          ...   
way     136079469  de:Messegelände (Berlin)       NaN          NaN  ...   
        196694606        de:CityCube Berlin    Berlin           DE  ...   
        680746621                       NaN    Berlin          NaN  ...   
        719414532                       NaN       NaN          NaN  ...   

                  addr:postcode  addr:street addr:suburb building  check_date  \
element id                                                                      
way     136079469           NaN          NaN         NaN      NaN         NaN   
        196694606         14055    Messedamm     Westend      yes  2024-07-04   
        680746621         14055  Jafféstraße     Westend      yes         NaN   
        719414532           NaN          NaN         NaN      yes  2025-03-31   

                                  contact:website layer          short_name  \
element id                                                                    
way     136079469                             NaN   NaN                 NaN   
        196694606  http://www.citycube-berlin.de/     2  CityCube;City Cube   
        680746621                             NaN   NaN                 NaN   
        719414532                             NaN   NaN                 NaN   

                     addr:place         alt_name  
element id                                        
way     136079469           NaN              NaN  
        196694606           NaN              NaN  
        680746621  Messe Berlin  hub 27;Halle 27  
        719414532           NaN              NaN  

[4 rows x 21 columns]

### Check no Geometries missing

In [9]:
print("Missing geometries:", exhibition_centers_filtered.geometry.isna().sum())

Missing geometries: 0


### Add Latitude & Longitude columns to filtered geodataframe

In [10]:
# Reproject and extract lat/lon
exhibition_centers_filtered = exhibition_centers_filtered.to_crs(epsg=4326)
exhibition_centers_filtered['geometry'] = exhibition_centers_filtered['geometry'].apply(
    lambda geom: geom if geom.geom_type == 'Point' else geom.representative_point()
)
exhibition_centers_filtered['latitude'] = exhibition_centers_filtered.geometry.y
exhibition_centers_filtered['longitude'] = exhibition_centers_filtered.geometry.x

### Verify the lat/lon amounts are correct

In [11]:
print("Latitude range:", exhibition_centers_filtered["latitude"].min(), "to", exhibition_centers_filtered["latitude"].max())

print("Longitude range:", exhibition_centers_filtered["longitude"].min(), "to", exhibition_centers_filtered["longitude"].max())

Latitude range: 52.5001203 to 52.5392102
Longitude range: 13.267529130414474 to 13.570825810628019


### Reset index and change cand drop redundant element column

In [12]:
exhibition_centers_filtered = exhibition_centers_filtered.reset_index()


# Drop the redundant column "element"
exhibition_centers_filtered = exhibition_centers_filtered.drop(columns=["element"],errors='ignore')

# Show final list of columns
print(exhibition_centers_filtered.columns.tolist())

['id', 'geometry', 'amenity', 'landuse', 'name', 'website', 'wikidata', 'wikimedia_commons', 'wikipedia', 'addr:city', 'addr:country', 'addr:housenumber', 'addr:postcode', 'addr:street', 'addr:suburb', 'building', 'check_date', 'contact:website', 'layer', 'short_name', 'addr:place', 'alt_name', 'latitude', 'longitude']


### Set the id to string

In [13]:
exhibition_centers_filtered["id"] = exhibition_centers_filtered["id"].astype(str)

In [14]:
exhibition_centers_filtered.info(verbose=True)

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   id                 4 non-null      object  
 1   geometry           4 non-null      geometry
 2   amenity            4 non-null      object  
 3   landuse            1 non-null      object  
 4   name               4 non-null      object  
 5   website            2 non-null      object  
 6   wikidata           3 non-null      object  
 7   wikimedia_commons  1 non-null      object  
 8   wikipedia          2 non-null      object  
 9   addr:city          2 non-null      object  
 10  addr:country       1 non-null      object  
 11  addr:housenumber   1 non-null      object  
 12  addr:postcode      2 non-null      object  
 13  addr:street        2 non-null      object  
 14  addr:suburb        2 non-null      object  
 15  building           3 non-null      object  
 16  chec

## 🏢🔄 Step 2: Exhibition Center Data Transformation

### Drop unnecessary columns
    - Dont need Berlin and DE as user knows they are looking for data in Berlin, Germany
    - Drop amenity column as all Exhibition_center
    - Drop suburb as it is the same as neighborhood which will be added later to make sure nothing is missing
    - Short name as the same as full name
    - Layers, Wikidata and Wikipedie media, addr:place, alt_name, check_date, landuse as not needed

In [15]:
exhibition_centers_filtered.drop(columns=['addr:city', 'addr:country', 'amenity', 'addr:suburb', 'landuse', 'wikidata', 'wikimedia_commons', 'check_date', 'layer', 'short_name', 'addr:place', 'alt_name'], errors='ignore', inplace=True)


### Copy the filtered database for record purposes and then standardise column names

In [16]:
exhibition_centers_cleaned = exhibition_centers_filtered.copy()

def clean_column(col):
    col = col.strip().lower().replace('addr:', '')
    col = col.replace(' ', '_')
    col = re.sub(r'[^a-z0-9_]', '', col)
    return col

exhibition_centers_cleaned.columns = [clean_column(col) for col in exhibition_centers_cleaned.columns]

### Rename columns for clarity

In [17]:
exhibition_centers_cleaned.rename(columns={
    'postcode': 'postal_code',
    'housenumber': 'house_number',
    'name': 'exhibition_center_name',
    'contactphone': 'phone',
    'contactwebsite': 'contact_website'
}, inplace=True)

In [18]:
print(exhibition_centers_cleaned.columns.tolist())

['id', 'geometry', 'exhibition_center_name', 'website', 'wikipedia', 'house_number', 'postal_code', 'street', 'building', 'contact_website', 'latitude', 'longitude']


### Fetch Districts & Neighbourhoods
- Load official Berlin districts GeoDataFrame

In [19]:
districts_gdf = gpd.read_file("../sources/geojson_files/lor_ortsteile.geojson")

In [20]:
districts_gdf.head(2)

,gml_id,spatial_name,spatial_alias,spatial_type,OTEIL,BEZIRK,FLAECHE_HA,geometry
0,re_ortsteil.0101,0101,Mitte,Polygon,Mitte,Mitte,1063.8748,"POLYGON ((13.41649 52.52696, 13.41635 52.52702..."
1,re_ortsteil.0102,0102,Moabit,Polygon,Moabit,Mitte,768.7909,"POLYGON ((13.33884 52.51974, 13.33884 52.51974..."


### Reproject GeoDataFrames to EPSG:4326 

In [21]:
exhibition_centers_cleaned = exhibition_centers_cleaned.to_crs(epsg=4326)
districts_gdf = districts_gdf.to_crs(epsg=4326)

### Spatial join with District Name and Neighbourhood_id(spatial_name)

In [22]:
exhibition_centers_df_district = gpd.sjoin(
    exhibition_centers_cleaned,
    districts_gdf[["BEZIRK", "spatial_name","geometry"]],
    how="left",
    predicate="within"
)

### Rename columns for clarity

In [23]:
exhibition_centers_df_district = exhibition_centers_df_district.rename(columns={
    "BEZIRK": "district",
    "spatial_name": "neighborhood_id"
}).drop(columns=["index_right"])  # drop district_number if not needed

### District mapping (official codes as strings)

In [24]:
district_mapping = {
    'Mitte': '11001001',
    'Friedrichshain-Kreuzberg': '11002002',
    'Pankow': '11003003',
    'Charlottenburg-Wilmersdorf': '11004004',
    'Spandau': '11005005',
    'Steglitz-Zehlendorf': '11006006',
    'Tempelhof-Schöneberg': '11007007',
    'Neukölln': '11008008',
    'Treptow-Köpenick': '11009009',
    'Marzahn-Hellersdorf': '11010010',
    'Lichtenberg': '11011011',
    'Reinickendorf': '11012012'
}

# Apply mapping to create district_id column (string)
exhibition_centers_df_district['district_id'] = exhibition_centers_df_district['district'].map(district_mapping).astype(str)

In [25]:
exhibition_centers_df_district.head()

,id,geometry,exhibition_center_name,website,wikipedia,house_number,postal_code,street,building,contact_website,latitude,longitude,district,neighborhood_id,district_id
0,136079469,POINT (13.2722 52.50345),Messe Berlin,https://www.messe-berlin.de/,de:Messegelände (Berlin),NaN,NaN,NaN,NaN,NaN,52.503448,13.272199,Charlottenburg-Wilmersdorf,0405,11004004
1,196694606,POINT (13.27079 52.50012),CityCube Berlin,NaN,de:CityCube Berlin,26,14055,Messedamm,yes,http://www.citycube-berlin.de/,52.500120,13.270794,Charlottenburg-Wilmersdorf,0405,11004004
2,680746621,POINT (13.26753 52.5029),hub27,https://www.messe-berlin.de/de/veranstalter/un...,NaN,NaN,14055,Jafféstraße,yes,NaN,52.502896,13.267529,Charlottenburg-Wilmersdorf,0405,11004004
3,719414532,POINT (13.57083 52.53921),Ausstellungspavillon,NaN,NaN,NaN,NaN,NaN,yes,NaN,52.539210,13.570826,Marzahn-Hellersdorf,1001,11010010


### Used reverse Geocoding and Nominatim to get Postal code & Street info
    - Created new columns at end to check against nulls in original columns

In [26]:
tqdm.pandas()

# Initialize geocoder
geolocator = Nominatim(user_agent="museum_locator")

# Define function to extract postal code and street
def get_postcode_and_street(row):
    try:
        location = geolocator.reverse((row['latitude'], row['longitude']), exactly_one=True)
        address = location.raw.get('address', {})
        postcode = address.get('postcode')
        street = address.get('road') or address.get('pedestrian') or address.get('footway') or address.get('street')
        return pd.Series([postcode, street])
    except:
        return pd.Series([None, None])

# Apply to your DataFrame
exhibition_centers_df_district[['postal_code_from_geo', 'street_from_geo']] = exhibition_centers_df_district.progress_apply(get_postcode_and_street, axis=1)


100%|██████████| 4/4 [00:10<00:00,  2.67s/it]


In [27]:
exhibition_centers_full = exhibition_centers_df_district.copy()

In [28]:
exhibition_centers_full.head()

,id,geometry,exhibition_center_name,website,wikipedia,house_number,postal_code,street,building,contact_website,latitude,longitude,district,neighborhood_id,district_id,postal_code_from_geo,street_from_geo
0,136079469,POINT (13.2722 52.50345),Messe Berlin,https://www.messe-berlin.de/,de:Messegelände (Berlin),NaN,NaN,NaN,NaN,NaN,52.503448,13.272199,Charlottenburg-Wilmersdorf,0405,11004004,14052,Alemannenallee
1,196694606,POINT (13.27079 52.50012),CityCube Berlin,NaN,de:CityCube Berlin,26,14055,Messedamm,yes,http://www.citycube-berlin.de/,52.500120,13.270794,Charlottenburg-Wilmersdorf,0405,11004004,None,None
2,680746621,POINT (13.26753 52.5029),hub27,https://www.messe-berlin.de/de/veranstalter/un...,NaN,NaN,14055,Jafféstraße,yes,NaN,52.502896,13.267529,Charlottenburg-Wilmersdorf,0405,11004004,None,None
3,719414532,POINT (13.57083 52.53921),Ausstellungspavillon,NaN,NaN,NaN,NaN,NaN,yes,NaN,52.539210,13.570826,Marzahn-Hellersdorf,1001,11010010,None,None


### Check null count before replacing Nulls in postal code

In [29]:
print(exhibition_centers_full[['postal_code']].isnull().sum())
print(exhibition_centers_full[['street']].isnull().sum())

postal_code    2
dtype: int64
street    2
dtype: int64


### Replace the Nulls in the original columns

In [30]:
exhibition_centers_full['postal_code'] = exhibition_centers_full['postal_code'].fillna(exhibition_centers_full['postal_code_from_geo'])
exhibition_centers_full['street'] = exhibition_centers_full['street'].fillna(exhibition_centers_full['street_from_geo'])

### Check null values now to see if a difference

In [31]:
print(exhibition_centers_full[['postal_code']].isnull().sum())
print(exhibition_centers_full[['street']].isnull().sum())

postal_code    1
dtype: int64
street    1
dtype: int64


### Replace empty strings with NaN

In [32]:
exhibition_centers_full.replace('', np.nan, inplace=True)

exhibition_centers_full.head()

,id,geometry,exhibition_center_name,website,wikipedia,house_number,postal_code,street,building,contact_website,latitude,longitude,district,neighborhood_id,district_id,postal_code_from_geo,street_from_geo
0,136079469,POINT (13.2722 52.50345),Messe Berlin,https://www.messe-berlin.de/,de:Messegelände (Berlin),NaN,14052,Alemannenallee,NaN,NaN,52.503448,13.272199,Charlottenburg-Wilmersdorf,0405,11004004,14052,Alemannenallee
1,196694606,POINT (13.27079 52.50012),CityCube Berlin,NaN,de:CityCube Berlin,26,14055,Messedamm,yes,http://www.citycube-berlin.de/,52.500120,13.270794,Charlottenburg-Wilmersdorf,0405,11004004,None,None
2,680746621,POINT (13.26753 52.5029),hub27,https://www.messe-berlin.de/de/veranstalter/un...,NaN,NaN,14055,Jafféstraße,yes,NaN,52.502896,13.267529,Charlottenburg-Wilmersdorf,0405,11004004,None,None
3,719414532,POINT (13.57083 52.53921),Ausstellungspavillon,NaN,NaN,NaN,None,None,yes,NaN,52.539210,13.570826,Marzahn-Hellersdorf,1001,11010010,None,None


### Check if website NaN then use data from contact_website if available

In [33]:
exhibition_centers_full['website'] = exhibition_centers_full['website'].fillna(exhibition_centers_full['contact_website'])

### Normalize the street name column

In [34]:
def normalize_street_name(name):
    if pd.isna(name):
        return np.nan
    # Replace underscores with spaces
    name = name.replace('_', ' ').replace('-', ' ')
    # Replace 'str.' or 'str' at end with ' Straße'
    name = re.sub(r'\bstr\.?\s*$', ' Straße', name, flags=re.IGNORECASE)
    # Ensure space before 'straße' if missing
    name = re.sub(r'(?<!\s)(straße)$', r' Straße', name, flags=re.IGNORECASE)
    # Ensure space before 'allee' if missing
    name = re.sub(r'(?<!\s)(allee)$', r' Allee', name, flags=re.IGNORECASE)
    # Ensure space before 'damm' if missing
    name = re.sub(r'(?<!\s)(damm)$', r' Damm', name, flags=re.IGNORECASE)
    # Ensure space before 'weg' if missing
    name = re.sub(r'(?<!\s)(weg)$', r' Weg', name, flags=re.IGNORECASE)
    # Ensure space before 'graben' if missing
    name = re.sub(r'(?<!\s)(graben)$', r' Graben', name, flags=re.IGNORECASE)
    # Ensure space before 'ufer' if missing
    name = re.sub(r'(?<!\s)(ufer)$', r' Ufer', name, flags=re.IGNORECASE)
    # Ensure space before 'korso' if missing
    name = re.sub(r'(?<!\s)(korso)$', r' Korso', name, flags=re.IGNORECASE)
    # Ensure space before 'zeile' if missing
    name = re.sub(r'(?<!\s)(zeile)$', r' Ziele', name, flags=re.IGNORECASE)
    # Ensure space before 'promenade' if missing
    name = re.sub(r'(?<!\s)(promenade)$', r' Promenade', name, flags=re.IGNORECASE)
    # Ensure space before 'kiez' if missing
    name = re.sub(r'(?<!\s)(kiez)$', r' Kiez', name, flags=re.IGNORECASE)
    # Ensure space before 'platz' if missing
    name = re.sub(r'(?<!\s)(platz)$', r' Platz', name, flags=re.IGNORECASE)
    # Ensure space before 'steig' if missing
    name = re.sub(r'(?<!\s)(steig)$', r' Steig', name, flags=re.IGNORECASE)
    # Remove extra spaces
    name = re.sub(r'\s+', ' ', name).strip()
    return name

In [35]:
exhibition_centers_full['street'] = exhibition_centers_full['street'].apply(normalize_street_name)

### Drop unnecessary columns

In [36]:
exhibition_centers_full.drop(columns=['geometry', 'district', 'postal_code_from_geo', 'street_from_geo', 'contact_website'], errors='ignore', inplace=True)

### Convert all text in columns to lowercase to avoid any duplications

In [37]:
text_cols = ['exhibition_center_name', 'website', 'wikipedia', 'street', 'building']

for col in text_cols:
    if col in exhibition_centers_full.columns:
        exhibition_centers_full[col] = exhibition_centers_full[col].apply(
            lambda x: x.strip().lower() if isinstance(x, str) else x
        )

### Check column data types
    - All have correct type allocated so no need to change anything

In [38]:
exhibition_centers_full.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 4 entries, 0 to 3
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      4 non-null      object 
 1   exhibition_center_name  4 non-null      object 
 2   website                 3 non-null      object 
 3   wikipedia               2 non-null      object 
 4   house_number            1 non-null      object 
 5   postal_code             3 non-null      object 
 6   street                  3 non-null      object 
 7   building                3 non-null      object 
 8   latitude                4 non-null      float64
 9   longitude               4 non-null      float64
 10  neighborhood_id         4 non-null      object 
 11  district_id             4 non-null      object 
dtypes: float64(2), object(10)
memory usage: 416.0+ bytes


### Remove duplicates
    - In this case we have no duplicates but will keep this check for future runs incase

- See how many duplicate rows exist

In [39]:
exhibition_centers_full.duplicated().sum()

np.int64(0)

- Display the actual duplicate rows

In [40]:
exhibition_centers_full[exhibition_centers_full.duplicated()]

,id,exhibition_center_name,website,wikipedia,house_number,postal_code,street,building,latitude,longitude,neighborhood_id,district_id


- Remove duplicate rows and reset index

In [41]:
exhibition_centers_full = exhibition_centers_full.drop_duplicates().reset_index(drop=True)

### Reorder column names for clarity

In [42]:
exhibition_center_listings = exhibition_centers_full[['id', 'exhibition_center_name', 'house_number', 'street', 'neighborhood_id', 'district_id', 'postal_code',  'website', 'building', 'wikipedia', 'latitude', 'longitude']]

In [43]:
exhibition_center_listings.to_csv("../sources/csv_files/exhibition_center_listings.csv", index=False)

### Final Summary of cleaned and Transformed Data

In [44]:
print(f"Number of rows: {exhibition_center_listings.shape[0]}")
print(f"Number of columns: {exhibition_center_listings.shape[1]}")

Number of rows: 4
Number of columns: 12


In [45]:
print("\nRemaining columns:")
print(exhibition_center_listings.columns.tolist())


Remaining columns:
['id', 'exhibition_center_name', 'house_number', 'street', 'neighborhood_id', 'district_id', 'postal_code', 'website', 'building', 'wikipedia', 'latitude', 'longitude']


In [46]:
missing = exhibition_center_listings.isnull().sum()
print("\nMissing values after cleaning and transforming :")
print(missing)


Missing values after cleaning and transforming :
id                        0
exhibition_center_name    0
house_number              3
street                    1
neighborhood_id           0
district_id               0
postal_code               1
website                   1
building                  1
wikipedia                 2
latitude                  0
longitude                 0
dtype: int64


In [47]:
missing_percent = exhibition_center_listings.isnull().mean() * 100
print(missing_percent.sort_values(ascending=False))

house_number              75.0
wikipedia                 50.0
street                    25.0
postal_code               25.0
website                   25.0
building                  25.0
id                         0.0
exhibition_center_name     0.0
neighborhood_id            0.0
district_id                0.0
latitude                   0.0
longitude                  0.0
dtype: float64


In [48]:
exhibition_center_listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      4 non-null      object 
 1   exhibition_center_name  4 non-null      object 
 2   house_number            1 non-null      object 
 3   street                  3 non-null      object 
 4   neighborhood_id         4 non-null      object 
 5   district_id             4 non-null      object 
 6   postal_code             3 non-null      object 
 7   website                 3 non-null      object 
 8   building                3 non-null      object 
 9   wikipedia               2 non-null      object 
 10  latitude                4 non-null      float64
 11  longitude               4 non-null      float64
dtypes: float64(2), object(10)
memory usage: 516.0+ bytes


In [49]:
exhibition_center_listings.head()

,id,exhibition_center_name,house_number,street,neighborhood_id,district_id,postal_code,website,building,wikipedia,latitude,longitude
0,136079469,messe berlin,NaN,alemannen allee,0405,11004004,14052,https://www.messe-berlin.de/,NaN,de:messegelände (berlin),52.503448,13.272199
1,196694606,citycube berlin,26,messe damm,0405,11004004,14055,http://www.citycube-berlin.de/,yes,de:citycube berlin,52.500120,13.270794
2,680746621,hub27,NaN,jaffé straße,0405,11004004,14055,https://www.messe-berlin.de/de/veranstalter/un...,yes,NaN,52.502896,13.267529
3,719414532,ausstellungspavillon,NaN,NaN,1001,11010010,None,NaN,yes,NaN,52.539210,13.570826


In [50]:
exhibition_center_listings.to_dict(orient='records')  # In Python (To see how to change to JSON code for Elasticsearch)

[{'id': '136079469',
  'exhibition_center_name': 'messe berlin',
  'house_number': nan,
  'street': 'alemannen allee',
  'neighborhood_id': '0405',
  'district_id': '11004004',
  'postal_code': '14052',
  'website': 'https://www.messe-berlin.de/',
  'building': nan,
  'wikipedia': 'de:messegelände (berlin)',
  'latitude': 52.5034485,
  'longitude': 13.272198914085445},
 {'id': '196694606',
  'exhibition_center_name': 'citycube berlin',
  'house_number': '26',
  'street': 'messe damm',
  'neighborhood_id': '0405',
  'district_id': '11004004',
  'postal_code': '14055',
  'website': 'http://www.citycube-berlin.de/',
  'building': 'yes',
  'wikipedia': 'de:citycube berlin',
  'latitude': 52.5001203,
  'longitude': 13.270793601633809},
 {'id': '680746621',
  'exhibition_center_name': 'hub27',
  'house_number': nan,
  'street': 'jaffé straße',
  'neighborhood_id': '0405',
  'district_id': '11004004',
  'postal_code': '14055',
  'website': 'https://www.messe-berlin.de/de/veranstalter/unsere-l

## 🏢🗃️ Step 3: Populate Database

### Import extra Libraries

In [51]:
import psycopg2
from sqlalchemy import create_engine, text
import warnings

warnings.filterwarnings("ignore")

### Credentials

In [ ]:
user_name=''
password=''

### Create the connection

In [53]:
# Conection
host = ''
port = ''
database = ''
schema=''

#connection to db after you opened tunnel
engine = create_engine(f'postgresql+psycopg2://{user_name}:{password}@{host}:{port}/{database}')

### Use table tested above with contraints

In [54]:
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {schema}.exhibition_centers (
    id VARCHAR(20) PRIMARY KEY,   
    exhibition_center_name VARCHAR(200) NOT NULL,
    house_number VARCHAR(20),           
    street VARCHAR(200),
    neighborhood_id VARCHAR(20),
    district_id VARCHAR(200) NOT NULL,
    postal_code VARCHAR(20),
    website VARCHAR(250),
    building VARCHAR(200),              
    wikipedia VARCHAR(250),
    latitude DECIMAL(9,6) NOT NULL,
    longitude DECIMAL(9,6) NOT NULL,
    CONSTRAINT district_id_fk FOREIGN KEY (district_id)
        REFERENCES berlin_source_data.districts(district_id)
        ON DELETE RESTRICT
        ON UPDATE CASCADE
);
"""

### Execute the create table query

In [56]:
with engine.connect() as conn:
    conn.execute(text(create_table_query))
    conn.commit()  # commit the transaction
    print("Table 'exhibition_centers' created or already exists.")  

Table 'exhibition_centers' created or already exists.


### Write the table to the layered Database using to.sql()

In [57]:
exhibition_center_listings.to_sql(
    'exhibition_centers',      
    engine,
    schema=schema,
    if_exists='append', # ✅ keeps table, just inserts data
    index=False
)

print("DataFrame sent to PostgreSQL using .to_sql() with psycopg2!")

DataFrame sent to PostgreSQL using .to_sql() with psycopg2!


### Query the table with SQL

In [58]:
query = f"""
SELECT * 
FROM berlin_source_data.exhibition_centers
LIMIT 10;
"""

# Execute the query
with engine.connect() as conn:
    df= pd.read_sql(text(query), conn)
    conn.commit()  # commit the transaction
df

,id,exhibition_center_name,house_number,street,neighborhood_id,district_id,postal_code,website,building,wikipedia,latitude,longitude
0,136079469,messe berlin,None,alemannen allee,0405,11004004,14052,https://www.messe-berlin.de/,None,de:messegelände (berlin),52.503449,13.272199
1,196694606,citycube berlin,26,messe damm,0405,11004004,14055,http://www.citycube-berlin.de/,yes,de:citycube berlin,52.500120,13.270794
2,680746621,hub27,None,jaffé straße,0405,11004004,14055,https://www.messe-berlin.de/de/veranstalter/un...,yes,None,52.502896,13.267529
3,719414532,ausstellungspavillon,None,None,1001,11010010,None,None,yes,None,52.539210,13.570826


### Check all districts exist in the districts table

In [61]:
query = f"""
SELECT ec.district_id, di.district_id, COUNT(*) as center_count
FROM berlin_source_data.exhibition_centers ec
JOIN berlin_source_data.districts di
ON ec.district_id = di.district_id
GROUP BY ec.district_id, di.district_id
;
"""

# Execute the query
with engine.connect() as conn:
    df= pd.read_sql(text(query), conn)
    conn.commit()  # commit the transaction
df

,district_id,district_id,center_count
0,11010010,11010010,1
1,11004004,11004004,3
